# Load packages

In [ ]:
from core.DataLoader import *
import seaborn as sn
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

indexed_feature_names = ['A. left_cardinality', 'B. base_cardinality', 
                 'C. sel_on_indexed_attr', 'D. sel_on_non_indexed_attr', 
                 'E. sel_of_join_pred', 'F. left_ordered', 'G. base_ordered', 'H. left_indexed', 
                 'I. base_indexed', 'J. result_size', 'K. pred_op_num_on_indexed_attr', 
                 'L. pred_op_num_on_non_indexed_attr']
feature_indices = ['A.', 'B.', 'C.', 'D.', 'E.', 'F.', 'G.', 'H.', 'I', 'J.', 'K.' ,'L.']

# Correlations of features on Postgres

In [ ]:
engine = 'postgres'
dl = DataLoader(engine)
all_ds = dl.get_one_file_ds(datasets=['ssb'])


for i in range(len(all_ds)):
    ds = all_ds[i]
    features = dl.base_features
    classification_target = dl.classification_target
    
    X = ds[features]
    y = ds['optimal_decision']
    X_costs = ds[dl.regression_targets]
    corrMatrix = X.corr()
    data = corrMatrix.to_numpy()
    data = np.abs(data)

    data[data < 0.8] = 0
    df = pd.DataFrame(data, columns=feature_indices, index=indexed_feature_names)
    fig, ax = plt.subplots(figsize=(14,8))
    sn_hm = sn.heatmap(df, annot=False, ax=ax, cbar_kws={'label': 'Pearson Coefficient'}, cmap=plt.cm.get_cmap('Reds'))
    cax = plt.gcf().axes[-1]
    cax.tick_params(labelsize=20)
    
    cax = plt.gcf().axes[-2]
    cax.tick_params(labelsize=20)
    
    sn_hm.figure.axes[-1].yaxis.label.set_size(20)

    plt.tight_layout()
    
    # save the figure
    # plt.savefig(f'./figures/exp2-{engine}-feat-corr.pdf')
    
    plt.show()
    # remove break for more correlations
    break

# Correlations of features on Microsoft SQL Server

In [ ]:
engine = 'mssql'
dl = DataLoader(engine)
all_ds = dl.get_one_file_ds(datasets=['ssb'])


for i in range(len(all_ds)):
    ds = all_ds[i]
    features = dl.base_features
    classification_target = dl.classification_target
    
    X = ds[features]
    y = ds['optimal_decision']
    X_costs = ds[dl.regression_targets]
    corrMatrix = X.corr()
    data = corrMatrix.to_numpy()
    data = np.abs(data)

    data[data < 0.8] = 0
    df = pd.DataFrame(data, columns=indices, index=indexed_feature_names)
    fig, ax = plt.subplots(figsize=(14,8))
    sn_hm = sn.heatmap(df, ansnot=False, ax=ax, cbar_kws={'label': 'Pearson Coefficient'}, cmap=plt.cm.get_cmap('Reds'))
    
    
    cax = plt.gcf().axes[-1]
    cax.tick_params(labelsize=20)
    
    cax = plt.gcf().axes[-2]
    cax.tick_params(labelsize=20)
    
    sn_hm.figure.axes[-1].yaxis.label.set_size(20)
    
    plt.tight_layout()
    # save the figure
    # plt.savefig(f'./figures/exp2-{engine}-feat-corr.pdf')
    plt.show()
    # remove break for more correlations
    break